<a href="https://colab.research.google.com/github/SerSanC/Master-Degree-Artificial-Inteligent/blob/main/ProjectSupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Preprocessing

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import DataLoader
from google.colab import drive
import random
import shutil
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import helper

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/femaleeyes'

In [4]:
len(os.listdir(base_dir))
#Creamos nuevas carpetas para cats
test_female_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/test'
os.makedirs(test_female_eyes, exist_ok=True)

#Creamos nuevas carpetas para dogs
test_male_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test'
os.makedirs(test_male_eyes, exist_ok=True)

train_female_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/train'
os.makedirs(train_female_eyes, exist_ok=True)

#Creamos nuevas carpetas para dogs
train_male_eyes = '/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/train'
os.makedirs(train_male_eyes, exist_ok=True)

In [5]:
len(os.listdir(base_dir))

5203

In [6]:

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir)))* 0.7)):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/train/{fname}") 

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir))))):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/test/{fname}") 

In [8]:

for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir)))* 0.7)):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/train/{fname}") 



In [7]:
len(os.listdir(base_dir))

0

In [11]:
for fname in random.sample(os.listdir(base_dir),int((len(os.listdir(base_dir))))):
  shutil.move(f"{base_dir}/{fname}", f"/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test/{fname}") 

In [16]:
int((len(os.listdir("/content/drive/MyDrive/Colab Notebooks/Datos/archive/male_eyes_dataset/test/"))))

1898

In [12]:
len(os.listdir(base_dir))

0

In [8]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

base_dir = "/content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/"


transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])


dataset = datasets.ImageFolder(base_dir,transform = transform)

dataset

Dataset ImageFolder
    Number of datapoints: 5202
    Root location: /content/drive/MyDrive/Colab Notebooks/Datos/archive/female_eyes_dataset/
    StandardTransform
Transform: Compose(
               Resize(size=255, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [9]:
classes = dataset.classes
data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

In [10]:
from torch.utils.data import random_split

train_set, test_set = random_split(dataset, (int(len(dataset) * 0.7) + 1, int(len(dataset) * 0.3)))
train_set, valid_set = random_split(train_set, (int(len(train_set) * 0.7) + 1, int(len(train_set) * 0.3)))

train_set

In [11]:
train_loader = DataLoader(train_set, batch_size=64)
valid_loader = DataLoader(valid_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(46656, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [13]:
from torch.optim import Adam
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn = Net().to(device)

criterion = nn.CrossEntropyLoss()
# params = cnn.resnet.fc.parameters()
params = cnn.parameters()
optimizer = Adam(params, lr=0.003)

In [14]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, device):
  total_step = len(train_loader)
  num_epochs = 10

  for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    for i, (img, target) in enumerate(train_loader):
      img = img.to(device)
      target = target.to(device)

      optimizer.zero_grad()

      output = model(img)
      

      loss = criterion(output, target)

      loss.backward()
      optimizer.step()

      train_loss += loss.item() * img.size(0)

    model.eval()
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)
      valid_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))

In [15]:
model = train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)

Epoch: 0 	Training Loss: 0.751302 	Validation Loss: 0.866672
Epoch: 1 	Training Loss: 0.615403 	Validation Loss: 0.867364
Epoch: 2 	Training Loss: 0.613772 	Validation Loss: 0.866877
Epoch: 3 	Training Loss: 0.610276 	Validation Loss: 0.872598
Epoch: 4 	Training Loss: 0.609790 	Validation Loss: 0.871800
Epoch: 5 	Training Loss: 0.604625 	Validation Loss: 0.874482
Epoch: 6 	Training Loss: 0.591423 	Validation Loss: 0.887244
Epoch: 7 	Training Loss: 0.565462 	Validation Loss: 0.926811
Epoch: 8 	Training Loss: 0.527465 	Validation Loss: 1.071517
Epoch: 9 	Training Loss: 0.466590 	Validation Loss: 1.093759


In [41]:
def global_accuracy(model, test_loader):
  correct = 0
  total = 0
  model.to("cpu")
  dataiter = iter(test_loader)
  with torch.no_grad():
    for data in dataiter:
      img, label = data
      output = model(img)
      _, predicted = torch.max(output.data, 1)
      total += label.size(0)
      correct += (predicted == label).sum().item()

  print(f"Accuracy: {100 * correct / total}")

def accuracy_per_class(model, test_loader, classes, device):
  class_correct = list(0. for i in range(5))
  class_total = list(0. for i in range(5))
  cnn.to(device)
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          outputs = cnn(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          if(c.item()):
            class_correct[labels.item()] += 1
          class_total[labels.item()] += 1
  

  accuracy = 0
  for i in range(0,2):
    print
    if int(class_total[i]) == 0:
      accuracy = 0
    else:
      accuracy = float(class_correct[i] / class_total[i])
    print(f"{classes[i]} | Correct: {class_correct[i]} | Total: {class_total[i]}" +
          f" | Accuracy: {accuracy}")



In [42]:
global_accuracy(cnn, test_loader)

Accuracy: 59.23076923076923


In [43]:
accuracy_per_class(cnn, test_loader, classes, device)

test | Correct: 122.0 | Total: 459.0 | Accuracy: 0.2657952069716776
train | Correct: 802.0 | Total: 1101.0 | Accuracy: 0.7284287011807448


In [46]:
torch.save(cnn.state_dict(), "cnn.pt")

In [51]:

model = Net()
model.load_state_dict(torch.load('cnn.pt'))

<All keys matched successfully>

In [52]:
from torchvision import models,transforms

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.resnet = models.resnet34(pretrained=True)
    self.resnet.fc = nn.Linear(self.resnet.fc.in_features,
                               5)

  def forward(self, image):
    output = self.resnet(image)
    return output

In [53]:
from torch.optim import Adam

cnn = Classifier().to(device)

criterion = nn.CrossEntropyLoss()
params = cnn.resnet.fc.parameters()
optimizer = Adam(params, lr=0.003)

train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)
global_accuracy(cnn, test_loader)
accuracy_per_class(cnn, test_loader, classes, device)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Epoch: 0 	Training Loss: 0.695492 	Validation Loss: 0.882129
Epoch: 1 	Training Loss: 0.619963 	Validation Loss: 0.906040
Epoch: 2 	Training Loss: 0.606183 	Validation Loss: 0.920572
Epoch: 3 	Training Loss: 0.597417 	Validation Loss: 0.930184
Epoch: 4 	Training Loss: 0.590702 	Validation Loss: 0.937863
Epoch: 5 	Training Loss: 0.585424 	Validation Loss: 0.944340
Epoch: 6 	Training Loss: 0.581130 	Validation Loss: 0.950022
Epoch: 7 	Training Loss: 0.577546 	Validation Loss: 0.955125
Epoch: 8 	Training Loss: 0.574491 	Validation Loss: 0.959780
Epoch: 9 	Training Loss: 0.571840 	Validation Loss: 0.964071
Accuracy: 69.23076923076923
test | Correct: 18.0 | Total: 459.0 | Accuracy: 0.0392156862745098
train | Correct: 1062.0 | Total: 1101.0 | Accuracy: 0.9645776566757494
